In [1]:
import numpy as np
import matplotlib.image as mpimg  # for reading in images

import matplotlib.pyplot as plt
import cv2  # computer vision library

%matplotlib inline

In [10]:
import sys

In [13]:
# opencv loads the image in BGR, convert it to RGB
img = cv2.cvtColor(cv2.imread('left_hand_1.jpeg'), cv2.COLOR_BGR2RGB)
lower_white = np.array([100, 145, 145], dtype=np.uint8)
upper_white = np.array([255, 255, 255], dtype=np.uint8)
mask = cv2.inRange(img, lower_white, upper_white)  # could also use threshold
mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3)))  # "erase" the small white points in the resulting mask
mask = cv2.bitwise_not(mask)  # invert mask

# load background (could be an image too)
bk = np.full(img.shape,(0,130,0), dtype=np.uint8)  # background clr

# get masked foreground
fg_masked = cv2.bitwise_and(img, img, mask=mask)

# get masked background, mask must be inverted 
#mask = cv2.bitwise_not(mask)
#bk_masked = cv2.bitwise_and(bk, bk, mask=mask)

# combine masked foreground and masked background 
#final = cv2.bitwise_or(fg_masked, bk_masked)
#mask = cv2.bitwise_not(mask)  # revert mask to original

#final = cv2.cvtColor(final, cv2.COLOR_RGB2BGR)
#cv2.imshow("final", final )
#cv2.imwrite('greenImage.jpg',final)

#################
background_image=cv2.imread('1900x1200.jpg')
background_image=cv2.cvtColor(background_image,cv2.COLOR_BGR2RGB)

# to put gesture image on top-left corner, So create a ROI
rows,cols,channels = fg_masked.shape

rowsBg,colsBg,channelsBg = background_image.shape
if rowsBg<rows or colsBg<cols:
    sys.exit('Background image shape should be atleast as much as gesture image size.')

#background_image=background_image[0:rows,0:cols]  # if this line is removed, it takes the original size of the background
roi = background_image[0:rows, 0:cols ]

# Now create a mask of gesture image and create its inverse mask also
fg_maskedgray = cv2.cvtColor(fg_masked,cv2.COLOR_BGR2GRAY)
ret, mask = cv2.threshold(fg_maskedgray, 10, 255, cv2.THRESH_BINARY)
mask_inv = cv2.bitwise_not(mask)

# Now black-out the area of gesture image in ROI
background_image_bg = cv2.bitwise_and(roi,roi,mask = mask_inv)

# Take only region of gesture image from gesture image image.
fg_masked_fg = cv2.bitwise_and(fg_masked,fg_masked,mask = mask)

# Put gesture image in ROI and modify the main image
dst = cv2.add(background_image_bg,fg_masked_fg)
background_image[0:rows, 0:cols ] = dst
background_image = cv2.cvtColor(background_image, cv2.COLOR_RGB2BGR)

# Save the final image as jpg
cv2.imwrite('UpdatedImage.jpg',background_image)
#plt.imshow(background_image)


True